In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import MDAnalysis as mda

from joblib import Parallel, delayed
from tqdm import tqdm

from MDAnalysis.analysis.base import AnalysisFromFunction
from MDAnalysis.lib.mdamath import triclinic_vectors
from ase import Atoms
from dscribe.descriptors import SOAP

In [2]:
# import os
# os.chdir("/home/suman/Dibyendu/ML_Crystal")

In [3]:
class SOAPFromCoords:
    def __init__(self, n_max=8, l_max=8, sigma=0.30, r_cut=2.0):
        self.soap = SOAP(
            species=["H"],
            periodic=True,
            r_cut=r_cut,
            n_max=n_max,
            l_max=l_max,
            rbf="gto",
            sigma=sigma,
            average="off",
            dtype="float32",
        )

    def compute_frame(self, coords, cell):
        n_atoms = coords.shape[0]
        atoms = Atoms(
            symbols=["H"] * n_atoms,
            positions=coords,
            cell=cell,
            pbc=True
        )
        return self.soap.create(
            atoms,
            centers=np.arange(n_atoms),
            n_jobs=1
        )


In [4]:
def compute_soap_from_dump(
    dump_file,
    atom_sel="type 1",
    n_frames=100,
    soap_kwargs=None,
    n_jobs=-1,
    stride = 1
):
    """
    Compute SOAP descriptors from a LAMMPS dump using AnalysisFromFunction.

    Returns
    -------
    soap : np.ndarray
        Shape (n_frames, n_atoms, n_features)
    """

    if soap_kwargs is None:
        soap_kwargs = dict(n_max=8, l_max=8, sigma=0.30, r_cut=2.0)

    # ---- suppress noisy MDAnalysis warnings ----
    with warnings.catch_warnings():
        warnings.filterwarnings(
            "ignore", message="Guessed all Masses to 1.0"
        )
        warnings.filterwarnings(
            "ignore", message="Reader has no dt information"
        )

        u = mda.Universe(dump_file, format="LAMMPSDUMP")

    sel = u.select_atoms(atom_sel)

    # ---- extract coordinates ----
    coordinates = (
        AnalysisFromFunction(lambda ag: ag.positions.copy(), sel)
        .run(stop=n_frames)
        .results["timeseries"]
        .astype(np.float32)
    )

    # ---- extract triclinic boxes ----
    box = (
        AnalysisFromFunction(
            lambda ag: triclinic_vectors(ag.dimensions.copy()), u.atoms
        )
        .run(stop=n_frames)
        .results["timeseries"]
        .astype(np.float32)
    )

    soaper = SOAPFromCoords(**soap_kwargs)

    # ---- parallel SOAP ----
    soap = Parallel(n_jobs=n_jobs)(
        delayed(soaper.compute_frame)(coordinates[i], box[i])
        for i in tqdm(range(0, len(coordinates), stride), desc="SOAP frames")
    )

    return np.asarray(soap, dtype=np.float32)


In [11]:
dump_file = "./assets/crystaltraj_dump5050_T_0.9.atom"

soap = compute_soap_from_dump(
    dump_file,
    atom_sel="type 1",
    n_frames=100,
    soap_kwargs=dict(n_max=8, l_max=8, sigma=0.30, r_cut=5.0),
    n_jobs=1, stride= 1
)

print(soap.shape), print(soap.dtype)

SOAP frames: 100%|██████████| 40/40 [02:46<00:00,  4.16s/it]

(40, 8000, 324)
float32


(None, None)

In [12]:
np.save("assets/test2npy", soap)